In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
# B---CIBLE
N_ETATS = 5   
ACTIONS = ['gauche', 'droite']     
EPSILON = 0.9  
ALPHA = 0.1  
GAMMA = 0.7   
MAX_CYCLES = 5   
TPS_MOUV = 0.5 

In [3]:
def Construire_q_table(n_etats, actions):
    table = pd.DataFrame(
        np.zeros((n_etats, len(actions))),     
        columns=actions,    
    )
    return table

In [4]:
def choix_action(etat, q_table):
    etat_actions = q_table.iloc[etat, :]
    if (np.random.uniform() > EPSILON) or ((etat_actions == 0).all()): 
        nom_action = np.random.choice(ACTIONS)
    else:   
        nom_action = etat_actions.idxmax()    
    return nom_action

In [5]:
def nouv_etat_r(S, A):
    if A == 'droite':    
        if S == N_ETATS - 2:  
            S_ = 'cible'
            R = 1
        else:
            S_ = S + 1
            R = 0
    else:   
        R = 0
        if S == 0:
            S_ = S  
        else:
            S_ = S - 1
    return S_, R

In [6]:
def maj_env(S, cycle, c_etape):
    env_list = ['-']*(N_ETATS-1) + ['Cible']  
    if S == 'cible':
        interaction = 'Cycle %s: nombre_pas = %s' % (cycle+1, c_etape)
        print('\r{}'.format(interaction), end='')
        time.sleep(2)
        print('\r                                ', end='')
    else:
        env_list[S] = 'B'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction), end='')
        time.sleep(TPS_MOUV)

In [7]:
def rl():
    q_table = Construire_q_table(N_ETATS, ACTIONS)
    for cycle in range(MAX_CYCLES):
        c_etape = 0
        S = 0
        fin_cycle = False
        maj_env(S, cycle, c_etape)
        while not fin_cycle:

            A = choix_action(S, q_table)
            S_, R = nouv_etat_r(S, A) 
            q_pred = q_table.loc[S, A]
            if S_ != 'cible':
                q_cible = R + GAMMA * q_table.iloc[S_, :].max()   
            else:
                q_cible = R    
                fin_cycle = True   

            q_table.loc[S, A] += ALPHA * (q_cible - q_pred)  
            S = S_  

            maj_env(S, cycle, c_etape+1)
            c_etape += 1
    return q_table

In [ ]:
if __name__ == "__main__":
    q_table = rl()
    print('\r\nQ-table:\n')
    print(q_table)

B---Cible                       